In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from my_functions import *
import pandas as pd

In [ ]:
tcurves = load_tcurves(load_filter_tags())
w_central = central_wavelength(tcurves)

In [ ]:
## Load STAR catalog
my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Star_jpas_mock_flam_1sq.cat')

sta_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
sta_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]

## Load GALAXY catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Gal_jpas_mock_flam_1sq.cat')

gal_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
gal_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]

## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_1sq.cat')

qso_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
qso_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]

In [ ]:
N_nb = 6
ew0min = 30

In [ ]:
nb_c_min = 6
nb_c_max = 50

N_gal = gal_flx.shape[1]
N_qso = qso_flx.shape[1]
N_sta = sta_flx.shape[1]

line_gal = np.zeros((nb_c_max - nb_c_min, N_gal)).astype(bool)
line_qso = np.zeros((nb_c_max - nb_c_min, N_qso)).astype(bool)
line_sta = np.zeros((nb_c_max - nb_c_min, N_sta)).astype(bool)

i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c, tcurves=tcurves)
    cont_est_qso, cont_err_qso = stack_estimation(qso_flx, qso_err, nb_c, N_nb, False)
    cont_est_gal, cont_err_gal = stack_estimation(gal_flx, gal_err, nb_c, N_nb, False)
    cont_est_sta, cont_err_sta = stack_estimation(sta_flx, sta_err, nb_c, N_nb, False)
    line_qso[i] = (
        (qso_flx[nb_c] - cont_est_qso > 3 * (cont_err_qso**2 + qso_err[nb_c]**2) ** 0.5)
        & (qso_flx[nb_c] - cont_est_qso > ew0min * (1 + z_nb) * cont_est_qso / fwhm)
        & (qso_err[nb_c] / qso_flx[nb_c] > 0.01)
    )
    line_gal[i] = (
        (gal_flx[nb_c] - cont_est_gal > 3 * (cont_err_gal**2 + gal_err[nb_c]**2) ** 0.5)
        & (gal_flx[nb_c] - cont_est_gal > ew0min * (1 + z_nb) * cont_est_gal / fwhm)
        & (gal_err[nb_c] / gal_flx[nb_c] > 0.01)
    )
    line_sta[i] = (
        (sta_flx[nb_c] - cont_est_sta > 3 * (cont_err_sta**2 + sta_err[nb_c]**2) ** 0.5)
        & (sta_flx[nb_c] - cont_est_sta > ew0min * (1 + z_nb) * cont_est_sta / fwhm)
        & (sta_err[nb_c] / sta_flx[nb_c] > 0.01)
    )
    i += 1

In [ ]:
# Arr where the center of the Lya line is. -1 means no lines detected
line_lya_center_qso = np.ones(N_qso, dtype=int) * -1
single_line_qso = 0

for mock_n in range(N_qso):
    try:
        first = np.where(line_qso[:, mock_n])[0][0] # First True in line_qso
    except:
        continue
    if first > 40: # We don't expect lines past here
        continue

    line_nb_width = 0
    while line_qso[first + line_nb_width + 1, mock_n]:
        line_nb_width += 1

    line_lya_center_qso[mock_n] = np.argmax(
        qso_flx[first + nb_c_min : first + nb_c_min + line_nb_width + 1, mock_n]
    ) + first + nb_c_min

    single_line_qso += int(np.all(~line_qso[first + 1 + line_nb_width:, mock_n]))

# Arr where the center of the Lya line is. -1 means no lines detected
line_lya_center_gal = np.ones(N_gal, dtype=int) * -1
single_line_gal = 0

for mock_n in range(N_gal):
    try:
        first = np.where(line_gal[:, mock_n])[0][0] # First True in line_qso
    except:
        continue
    if first > 40: # We don't expect lines past here
        continue

    line_nb_width = 0
    while line_gal[first + line_nb_width + 1, mock_n]:
        line_nb_width += 1

    line_lya_center_gal[mock_n] = np.argmax(
        gal_flx[first + nb_c_min : first + nb_c_min + line_nb_width + 1, mock_n]
    ) + first + nb_c_min

    single_line_gal += int(np.all(~line_gal[first + 1 + line_nb_width:, mock_n]))

# Arr where the center of the Lya line is. -1 means no lines detected
line_lya_center_sta = np.ones(N_sta, dtype=int) * -1
single_line_sta = 0

for mock_n in range(N_sta):
    try:
        first = np.where(line_sta[:, mock_n])[0][0] # First True in line_qso
    except:
        continue
    if first > 40: # We don't expect lines past here
        continue

    line_nb_width = 0
    while line_sta[first + line_nb_width + 1, mock_n]:
        line_nb_width += 1

    line_lya_center_sta[mock_n] = np.argmax(
        sta_flx[first + nb_c_min : first + nb_c_min + line_nb_width + 1, mock_n]
    ) + first + nb_c_min

    single_line_sta += int(np.all(~line_sta[first + 1 + line_nb_width:, mock_n]))

In [ ]:
print('TOTAL SELECTED')
print('Star  : {} / {}'
    .format(len(np.where(np.sum(line_sta.astype(int), axis=0) != 0)[0]), N_sta))
print('Galaxy: {} / {}'
    .format(len(np.where(np.sum(line_gal.astype(int), axis=0) != 0)[0]), N_gal))
print('QSO   : {} / {}'
    .format(len(np.where(np.sum(line_qso.astype(int), axis=0) != 0)[0]), N_qso))

print()
print('SINGLE LINE')
print('Star  : {} / {}'
    .format(single_line_sta, N_sta))
print('Galaxy: {} / {}'
    .format(single_line_gal, N_gal))
print('QSO   : {} / {}'
    .format(single_line_qso, N_qso))